# Computerphysik Programmiertutorial 10
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**Github**: [https://github.com/markusschmitt/compphys2022](https://github.com/markusschmitt/compphys2022)

**Inhalt dieses Notebooks**: Pseudo-Zufallszahlen generieren, Reproduzierbare Pseudo-Zufallszahlen, Qualität von Pseudo-Zufallszahlen, Gleichverteilung auf der Kugeloberfläche

# Pseudo-Zufallszahlen generieren

Die Erzeugung von Pseudo-Zufallszahlen ist Teil der Standardfunktionalität von Julia: [Dokumentation](https://docs.julialang.org/en/v1/stdlib/Random/).

Die `rand()`-Funktion erzeugt gleichverteilte reelle Zufallszahlen auf dem Intervall $[0,1]$.

Es können auch Vektoren von Zufallszahlen erzeugt werden:

Mit der `Plots` Funktion `hist()` können wir ein Histogramm der erzeugten Zufallszahlen plotten:

Ebenso können Matrizen mit Zufallszahlen gefüllt werden:

Anzeigen der Zufallsmatrix mit `heatmap()`:

## Ganzzahlige Zufallszahlen

Oft sind wir an ganzzahligen Zufallszahlen interessiert, z.B. wenn wir das Ergebnis eines Würfelwurfs simulieren wollen.

Solche Zufallszahlen können wir auch aus den $x\in[0,1]$ generieren, die `rand()` erzeugt:

Histogramm vieler Wiederholungen des Würfelwurfs:

## Bernoulli-Verteilung

Häufig tauchen auch Bernoulli-Zufallsvariablen auf, also Zufallsvariablen $X$, die zwei verschiedene Werte annehmen können. Bezeichnen wir diese Werte als $0$ und $1$, dann gilt

$$
\begin{aligned}
P(X=0)&=p\\
P(X=1)&=1-p
\end{aligned}
$$

Die Verteilung ist also durch den Parameter $p$ vollständig beschrieben.

Ein Beispiel ist der faire Münzwurf, für den $p=1/2$. Diesen können wir verallgemeinern zum Wurf einer gezinkten Münze mit beliebigen $p\in[0,1]$. Auch solche Zufallszahlen können wir basierend auf der `rand()`-Funktion implementieren:

Histogramm einer großen Zahl fairer Münzwürfe:

Histogramm einer großen Zahl gezinkter Münzwürfe:

## Gauß-Normalverteilung

Die Funktion `randn()` erzeugt normalverteilte Zufallszahlen:

In [ ]:
histogram(gauss,bins=100,normed=true);

xrange=range(-3,3,length=100)
plot!(xrange, exp.(-0.5*xrange.^2)/sqrt(2pi))

# Reproduzierbare Pseudo-Zufallszahlen: Seeds

Die Reproduzierbarkeit von Ergebnissen ist für wissenschaftliche Anwendungen essenziell. Bei Pseudo-Zufallszahlen-Generatoren kann der **seed** benutzt werden um identische Zufallszahlen zu reproduzieren.

Für diesen Zweck verwenden wir das Julia Packet `Random`.

Der seed des Zufallszahlengenerators ist eine ganze Zahl, die mit der Funktion `seed!()` festgelegt wird:

Erzeuge eine Folge von Zufallszahlen

Noch einmal so viele andere Zufallszahlen:

Wenn wir jetzt den seed wieder auf den gleichen Wert setzen wie oben, können wir die Folge der Zufallszahlen reproduzieren:

# Pseudo-Zufallszahlen generieren

Pseudo-Zufallszahlen (= zufällig aussehende Zahlenfolgen) können durch einfache iterative Vorschriften erzeugt werden.

## Linearer Kongruenzgenerator

Ein einfaches Beispiel ist der *lineare Kongruenzgenerator* mit der Iterationsvorschrift

$$
x_{n+1} = (ax_n+c)\ \text{mod}\ m
$$

Hier ist $x_n$ ein Element unserer Folge von Zufallszahlen, und $a$, $c$ und $m$ sind positive ganze Zahlen. Die Qualität der erzeugten Zufallszahlen hängt von der Wahl der Parameter ab. Eine gute Wahl ist

In [ ]:
a = 16807
c = 1
m = 2^31-1;

Implementieren wir damit also unseren eigenen linearen Kongruenzgenerator:

Erster Test:

Test mit bestimmtem seed:

Wir produzieren hier zufällige ganze Zahlen. Wie können wir zufällige Fließkommazahlen erzeugen?

## Inkrementeller Generator

Hier implementieren wir noch einen weiteren Generator von Zufallszahlen für den Vergleich im nächsten Abschnitt:

In [ ]:
function inkrementeller_generator(n=1)
    
    random_numbers = [0.05*rand()]
    for j in 1:n-1
        
        x = random_numbers[j] + 0.05*rand()
        
        if x > 1
            x = x-1
        end
        push!(random_numbers, x)
    end
    
    return random_numbers
end

# Qualität von Zufallszahlen

Unterschiedliche Zufallszahlengeneratoren produzieren Zufallszahlen von unterschiedlicher Qualität. Das werden wir hier beleuchten.

## Histogramme

Schauen wir zunächst Histogramme der Samples unserer Verschiedenen Generatoren an.

**Linearer Kongruenzgenerator**

In [ ]:
histogram(mein_lkg_real(1000000), bins=100)

Julia's **Mersenne Twister**

In [ ]:
histogram(rand(1000000), bins=100)

**Inkrementeller Generator**

In [ ]:
histogram(inkrementeller_generator(1000000), bins=100)

## Korrelationen

Eine wichtige Eigenschaft guter Pseudo-Zufallszahlen ist, dass aufeinanderfolgende Zahlen unkorreliert sind. Die Korrelationen zwischen aufeinanderfolgenden Zahlen können wir mit folgender Größe quantifizieren:

$$
\chi=\langle x_ix_{i+1}\rangle-\langle x_i\rangle^2
$$

Schreiben wir eine Funktion, die für eine Folge von Zufallszahlen die Korrelationen $\chi$ berechnet:

Wie stark sind die aufeinanderfolgende Zahlen unserer verschiedenen Generatoren korreliert?

**Linearer Kongruenzgenerator**

**Mersenne Twister**

**Inkrementeller Generator**

# Spektraltests

Mit einem *Spektraltest* können wir noch genauer hinschauen. Dafür interpretieren wir drei aufeinanderfolgende Zahlen einer Folge von Pseudo-Zufallszahlen als Koordinaten in einem dreidimensionalen Raum:

In [ ]:
function spectral_test(random_numbers)
    x = Float64[]
    y = Float64[]
    z = Float64[]
    
    for i in 1:length(random_numbers)-2
        if random_numbers[i] < 0.001
            push!(x, random_numbers[i])
            push!(y, random_numbers[i+1])
            push!(z, random_numbers[i+2])
        end
    end
    
    return x, y, z
end

Spektraltest des linearen Kongruenzgenerators:

In [ ]:
zahlen = mein_lkg_real(10000000)

X, Y, Z = spectral_test(zahlen);

Schauen wir uns einen Plot davon an:

In [ ]:
plotlyjs()
scatter(X,Y,Z, markersize=0.5)

Spektraltest des Mersenne Twisters:

In [ ]:
zahlen = rand(10000000)

X, Y, Z = spectral_test(zahlen);

plotlyjs()
scatter(X,Y,Z, markersize=0.5)

# Beispiel: Gleichverteilung auf der Kugeloberfläche

<img src="https://www.biancahoegel.de/wissen/navigation/bilder/kugelkoord-def.png"></img>

Koordinaten auf der Kugeloberfläche ausgedrückt durch Polarwinkel $\theta$ und Azimuthalwinkel $\varphi$:

$$
\begin{aligned}
x &= \sin\varphi\sin\theta\\
y &= \cos\varphi\sin\theta\\
z &= \cos\theta
\end{aligned}
$$

**Idee:** Sample zufällige Winkel $\varphi\in[0,2\pi]$ und $\theta\in[0,\pi]$ und rechne anschließend in karthesische Koordinaten um:

In [ ]:
function rnd_sphere(n)
    X = Float64[]
    Y = Float64[]
    Z = Float64[]
    
    for j in 1:n
        phi = rand() * 2pi
        theta = rand() * pi
        
        push!(X, sin(phi) * sin(theta))
        push!(Y, cos(phi) * sin(theta))
        push!(Z, cos(theta))
    end
    
    return X, Y, Z
end

Plot zufällig generierter Punkte:

In [ ]:
X, Y, Z = rnd_sphere(1000)

plotlyjs()
scatter(X,Y,Z, markersize=0.5)

**Alternative:** Sample zufälligen Winkel $\varphi\in[0,2\pi]$ und gleichverteilte $z$-Koordinate $z\in[-1,1]$. Dann ergeben sich

$$
\begin{aligned}
x &= \sqrt{1-z^2}\sin\varphi\\
y &= \sqrt{1-z^2}\cos\varphi
\end{aligned}
$$

In [ ]:
function rnd_sphere_uniform(n)
    X = Float64[]
    Y = Float64[]
    Z = Float64[]
    
    for j in 1:n
        phi = rand() * 2pi
        z = -1 + 2*rand()
        
        push!(X, sqrt(1-z^2) * cos(phi))
        push!(Y, sqrt(1-z^2) * sin(phi))
        push!(Z, z)
    end
    
    return X, Y, Z
end

In [ ]:
X, Y, Z = rnd_sphere_uniform(1000);

plotlyjs()
scatter(X,Y,Z, markersize=0.5)